In [2]:
! ls train_data -l --block-size=M

total 1519M
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh 408M Jan 24 12:45 2016-12-26.csv
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh 385M Jan 24 12:56 2017-01-02.csv
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh 366M Jan 24 13:05 2017-01-09.csv
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh 361M Jan 24 13:15 2017-01-16.csv


In [3]:
! ls test_data -l --block-size=M

total 360M
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh 360M Jan 24 14:25 2017-01-23.csv


In [4]:
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, metrics, feature_extraction
import numpy as np
import os
import json

pd.set_option('display.max_columns', 500)

In [5]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd
import plotly

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)



def plotly_df(df, title = '', filename = None):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link = False)
    if filename:
        plotly.offline.plot(fig, filename=filename)

1.12.11


## Подготовка данных

In [6]:
%%time
sample = 0.01

dfs_sampled = []

for filename in os.listdir('./train_data'):
    df = pd.read_csv('./train_data/' + filename).fillna(0)
    pos_df = df[df.target == 1]
    neg_df = df[df.target == 0]

    neg_df_sampled = neg_df.sample(frac = sample)
    df_sampled = pd.concat([pos_df, neg_df_sampled])
    dfs_sampled.append(df_sampled) 

df = pd.concat(dfs_sampled).drop_duplicates()
X = df.drop(['target', 'user_id'], axis = 1)
y = df.target

print X.shape

(X_train, X_test, y_train, y_test) = model_selection.train_test_split(X, y, 
                                                              test_size = 0.3, 
                                                              random_state = 0, 
                                                              stratify = y)

scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

(133898, 50)
CPU times: user 56.1 s, sys: 14.6 s, total: 1min 10s
Wall time: 1min 10s


#### Выделение данных Logs API

In [7]:
logs_api_cols = ['is_mobile',
 'is_bounce_last_visit',
 'days_since_last_visit',
 'days_since_first_visit',
 'is_yabrowser',
 'mobile',
 'first_source_direct',
 'first_source_referral',
 'first_source_organic',
 'first_source_ad',
 'first_source_saved',
 'first_source_undefined',
 'first_source_external',
 'first_source_email',
 'first_source_social',
 'first_source_internal',
 'avg_depth',
 'avg_duration',
 'visits',
 'not_bounce_visits',
 'purchases',
 'purchased_products',
 'viewed_products',
 'cart_products',
 'revenue',
 'ad_visits',
 'direct_visits',
 'email_visits',
 'internal_visits',
 'organic_visits',
 'referral_visits',
 'saved_visits',
 'social_visits',
 'undefined_visits']

In [8]:
X_api = df[logs_api_cols]
y_api = df.target

(X_train_api, X_test_api, y_train_api, y_test_api) = model_selection.train_test_split(X_api, y_api, 
                                                              test_size = 0.3, 
                                                              random_state = 0, 
                                                              stratify = y)

scaler = preprocessing.StandardScaler()
X_train_scaled_api = scaler.fit_transform(X_train_api)
X_test_scaled_api = scaler.transform(X_test_api)

## Обучение (XGBoost)

In [9]:
import xgboost as xgb

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [10]:
clf_xgb = xgb.XGBClassifier(
    n_estimators = 1000,
    colsample_bytree=1,
    learning_rate=0.025,
    max_depth=3,
    min_child_weight=10,
    subsample=0.5
)
%time clf_xgb.fit(X_train_scaled_api, y_train_api)

CPU times: user 2min 14s, sys: 212 ms, total: 2min 14s
Wall time: 17.8 s


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.025, max_delta_step=0, max_depth=3,
       min_child_weight=10, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.5)

In [11]:
xgb_features_df = pd.DataFrame()
xgb_features_df['weight'] = clf_xgb.feature_importances_
xgb_features_df['feature'] = logs_api_cols
xgb_features_df.sort_values('weight', ascending = False).head(10)

,weight,feature
3,0.159943,days_since_first_visit
17,0.135676,avg_duration
2,0.117554,days_since_last_visit
16,0.103057,avg_depth
22,0.083990,viewed_products
24,0.047274,revenue
19,0.039080,not_bounce_visits
18,0.039080,visits
25,0.031358,ad_visits
26,0.030098,direct_visits


In [12]:
y_xgb = clf_xgb.predict_proba(X_test_scaled_api)[:, 1]
roc_auc_xgb = metrics.roc_auc_score(y_test_api, y_xgb)
roc_auc_xgb 

0.86153358675776759

In [13]:
y_xgb_df = pd.DataFrame(y_xgb)
y_xgb_df.columns = ['probability']
y_xgb_df['probability_rounded'] = map(lambda x: 100*round(x, 2), y_xgb_df.probability)
plotly_df(y_xgb_df.groupby('probability_rounded').count())

## Предсказание для новых данных

In [14]:
%%time

dfs_= []

for filename in os.listdir('./test_data'):
    if filename.startswith('.'):
        continue
    df = pd.read_csv('./test_data/' + filename).fillna(0)
    dfs_sampled.append(df) 

df_exp = pd.concat(dfs_sampled).drop_duplicates()
print df_exp.shape

X_api_exp = df_exp[logs_api_cols]

X_scaled_api_exp = scaler.transform(X_api_exp)

(2976722, 52)
CPU times: user 23.9 s, sys: 6.88 s, total: 30.7 s
Wall time: 30.7 s


In [15]:
df_exp['prediction'] = clf_xgb.predict_proba(X_scaled_api_exp)[:, 1]

In [16]:
df_exp['prediction_rounded'] = map(lambda x: round(x, 2), df_exp.prediction)

In [17]:
predictions_df = df_exp.groupby('prediction_rounded')[['user_id']].count()

In [18]:
plotly_df(predictions_df, title = 'Распределение по вероятности')

In [19]:
plotly_df(predictions_df.apply(lambda x: 100.*x/predictions_df.user_id.sum()), title = 'Распределение по вероятности')

In [20]:
plotly_df(predictions_df.apply(lambda x: 100.*x/predictions_df.user_id.sum()).cumsum(), title = 'График с накоплением')

In [31]:
df_exp.to_csv('predictions_2017.csv')

In [32]:
! ls -l --block-size=M

total 457M
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh   1M Jan 24 13:33 get_users_data_client.ipynb
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh   1M Jan 24 15:38 ml_model.ipynb
-rw-r--r-- 1 miptgirl dpt_yandex_monetize_metrica_dev_beh 457M Jan 24 15:38 predictions_2017.csv
drwxr-xr-x 3 miptgirl dpt_yandex_monetize_metrica_dev_beh   1M Jan 24 15:16 test_data
drwxr-xr-x 2 miptgirl dpt_yandex_monetize_metrica_dev_beh   1M Jan 24 13:05 train_data


In [38]:
df_exp.shape

(2976723, 54)

In [32]:
df_exp['prediction_rounded_2'] = map(lambda x:  x - x % 0.05, df_exp.prediction)

In [44]:
pred_share = df_exp.groupby('prediction_rounded_2')[['user_id']].count()

In [45]:
pred_share['volume'] = map(lambda x: str(round(100.*x/pred_share.user_id.sum(), 2)) + '%', pred_share.user_id)

In [46]:
pred_share.reset_index(inplace = True)
pred_share.columns = ['predicted_probability', 'users', 'share by users']

In [48]:
pred_share.set_index('predicted_probability')

,users,share by users
predicted_probability,,
0.00,1804781,60.63%
0.05,520039,17.47%
0.10,231115,7.76%
0.15,124030,4.17%
0.20,77585,2.61%
0.25,52111,1.75%
0.30,37763,1.27%
0.35,28337,0.95%
0.40,22516,0.76%
